In [10]:
!pip install pycaret
# 한글 폰트 사용을 위한 라이브러리입니다.
!apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.


In [11]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [12]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from pycaret.regression import *
import pandas as pd

# 필요한 데이터를 load
total_path = "../total-data_ver2.csv"

dt_total = pd.read_csv(total_path)

# is_test 컬럼 값이 0인 행을 train 데이터셋으로 저장
df_train = dt_total[dt_total['is_test'] == 0]

# is_test 컬럼 값이 1인 행을 test 데이터셋으로 저장
df_test = dt_total[dt_total['is_test'] == 1]

In [13]:
# '계약년월'에서 '연도' 추출 후 int 형으로 변환하여 새로운 열 추가
df_train = df_train.assign(year=df_train['계약년월'].astype('str').str[:4].astype(int)).query('year > 2020')

# '계약년월'에서 '연도' 추출 후 int 형으로 변환하여 새로운 열 추가
df_test = df_test.assign(year=df_test['계약년월'].astype('str').str[:4].astype(int))

# '계약년월'에서 '월' 추출 후 5로 나눈 값을 새로운 열로 추가
df_train, df_test = map(lambda df: df.assign(계약월=lambda x: (x['계약년월'] % 100) // 5), [df_train, df_test])

# 타겟 변수 생성
df_train['price'] = df_train.groupby(['아파트명', '계약월'])['target'].transform('mean')

# 중복 데이터 제거
df_train = df_train.drop_duplicates(subset=['아파트명', '계약월'])

In [14]:
# 특징 변수와 타겟 변수 설정
X = df_train[['전용면적', 'x']]
y = df_train['price']

# 데이터를 학습 데이터와 검증 데이터로 분리합니다.
# 데이터 전처리 및 모델 학습
reg = setup(data = df_train, target = 'price', numeric_features = ['전용면적', 'x'], verbose=False)

# 모델 생성
best_model = compare_models()

# 모델 학습
final_model = finalize_model(best_model)

# 테스트 데이터에 대한 예측
pred = predict_model(final_model, data = df_test)

# 예측 결과 저장
pd.DataFrame(pred['prediction_label'].values.astype(int), columns=['target']).to_csv('simple.csv', index=False)



,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,24361.3075,2473025086.1030,48920.5001,0.6457,0.4469,0.3812,2.1220
lightgbm,Light Gradient Boosting Machine,24916.0208,2577531298.4561,49953.1921,0.6303,0.4577,0.3893,4.0610
xgboost,Extreme Gradient Boosting,25375.4154,2703379136.0000,51242.5117,0.6101,0.4732,0.4149,0.1090
dummy,Dummy Regressor,54338.0426,6851704473.6000,82166.1695,-0.0037,0.8022,1.0235,0.0340


In [15]:
# print(pred.columns)

In [16]:
# 저장된 파일 확인
print(pd.read_csv('simple.csv'))

      target
0      59666
1      49447
2      47377
3      47573
4      48399
...      ...
9267   72693
9268   74107
9269  106591
9270   98905
9271   94572

[9272 rows x 1 columns]
